In [1]:
from dotenv import load_dotenv # pip install python-dotenv
import os

load_dotenv()
parser_key = os.getenv("llama_parser")#It only parse 1000 pages per day for free or you will have to upgrade the plan 🤑💸
'''https://cloud.llamaindex.ai/api-key'''#and there must be 1200 pages per file max
pinecone_key=os.getenv("pinecone_api_key")
huggingface_key=os.getenv("HUGGINGFACE_TOKEN")
if parser_key:
    print(True)
if pinecone_key:
    print(True)
if huggingface_key:
    print(True)


True
True
True


In [2]:
import nest_asyncio
from llama_parse import LlamaParse


# Apply nest_asyncio to handle asynchronous operations
nest_asyncio.apply()

# Initialize LlamaParse with API key from environment variable
api_key = parser_key  # Ensure your API key is set here or in the environment variables
parser = LlamaParse(
    api_key=api_key,
    result_type="markdown",  # "markdown" and "text" are available
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
)

# Function to handle loading of documents synchronously
def extract_documents_sync(path):
    if os.path.isfile(path):
        return parser.load_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return parser.load_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Function to handle loading of documents asynchronously
async def extract_documents_async(path):
    if os.path.isfile(path):
        return await parser.aload_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return await parser.aload_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Path to the file or directory containing the documents
path = r"C:\Users\Sayan Maity\OneDrive\Desktop\chat_pdf\Research_on_Machine_Learning_and_Its_Algorithms_an.pdf"

# Extract and print the content of the documents synchronously
documents = extract_documents_sync(path)
for doc in documents:
    print(doc.text[:1000])# Print the first 1000 characters of each 




Started parsing the file under job_id 28b9e3df-ec31-4130-abd9-36cbd7358cc7
# Journal of Physics: Conference Series

PAPER &#8226; OPEN ACCESS

Research on Machine Learning and Its Algorithms and Development

To cite this article: Wei Jin 2020 J. Phys.: Conf. Ser. 1544 012003

View the article online for updates and enhancements.

IOP ebooks

Bringing together innovative digital publishing with leading authors from the global scientific community.

Start exploring the collection-download the first chapter of every title for free.

This content was downloaded from IP address 158.46.154.149 on 03/06/2020 at 13:35
# Research on Machine Learning and Its Algorithms and Development

Wei Jin

Northwestern Polytechnical University Ming De College, Xi’an, Shaanxi, China

Abstract: This article analyzes the basic classification of machine learning, including supervised learning, unsupervised learning, and reinforcement learning. It combines analysis on common algorithms in machine learning, such 

In [13]:
#pip install tf-keras
import os
from dotenv import load_dotenv
from llama_parse import LlamaParse
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load environment variables
load_dotenv()

# Initialize LlamaParse
parser_key = os.getenv("llama_parser")
parser = LlamaParse(api_key=parser_key, result_type="markdown", num_workers=4, verbose=True)

# Function to extract documents
def extract_documents(path):
    if os.path.isfile(path):
        return parser.load_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return parser.load_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Extract documents
path = r"C:\Users\Sayan Maity\OneDrive\Desktop\chat_pdf\Research_on_Machine_Learning_and_Its_Algorithms_an.pdf"
documents = extract_documents(path)

# Initialize Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create and save FAISS index
db = FAISS.from_documents(documents, embeddings)
db.save_local("faiss_index")

# Load the saved index
loaded_db = FAISS.load_local("faiss_index", embeddings)

# Initialize Hugging Face pipeline for language model
model_name = "google/flan-t5-large"  # You can change this to a different model if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=pipe)

# Create prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer concise.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Set up retriever with contextual compression
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=loaded_db.as_retriever(search_kwargs={"k": 4})
)

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# Function to answer questions
def answer_question(query):
    result = qa_chain({"query": query})
    return result["result"], result["source_documents"]

# Example usage
query = "How many machine learning algorithms are there?"
answer, source_docs = answer_question(query)
print(f"Answer: {answer}")
print("\nSource Documents:")
for doc in source_docs:
    print(f"- {doc.metadata['source']}: {doc.page_content[:100]}...")

Started parsing the file under job_id 58f333d4-4132-4586-b981-7c527e39bb2e


AttributeError: 'Document' object has no attribute 'page_content'

pip install langchain langchain-huggingface faiss-cpu transformers torch dotenv llama-parse

In [15]:
import os
from dotenv import load_dotenv
from llama_parse import LlamaParse
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load environment variables
load_dotenv()

# Initialize LlamaParse
parser_key = os.getenv("llama_parser")
parser = LlamaParse(api_key=parser_key, result_type="markdown", num_workers=4, verbose=True)

# Function to extract documents
def extract_documents(path):
    if os.path.isfile(path):
        return parser.load_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return parser.load_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Extract documents
path = r"C:\Users\Sayan Maity\OneDrive\Desktop\chat_pdf\Research_on_Machine_Learning_and_Its_Algorithms_an.pdf"
documents = extract_documents(path)

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

# Split documents into smaller chunks
texts = []
metadatas = []
for doc in documents:
    chunks = text_splitter.split_text(doc.text)
    texts.extend(chunks)
    metadatas.extend([doc.metadata] * len(chunks))

# Initialize Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create and save FAISS index
db = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
db.save_local("faiss_index")

# Load the saved index
loaded_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)



Started parsing the file under job_id 07cdd29f-4ddc-4d0b-859e-4e78ee0e9e80


In [17]:
# ... (previous code remains the same)

# Initialize Hugging Face pipeline for language model
model_name = "google/flan-t5-large"  # Upgraded to a larger model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=pipe)

# Create a more detailed prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Provide a detailed explanation in your answer, using up to five sentences.

Context: {context}

Question: {question}
Detailed Answer:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Set up retriever with contextual compression
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=loaded_db.as_retriever(search_kwargs={"k": 6})  # Increased from 4 to 6
)

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# Function to answer questions
def answer_question(query):
    result = qa_chain.invoke({"query": query})
    return result["result"], result["source_documents"]

# Example usage
query = "Can you summerize the topic?"
answer, source_docs = answer_question(query)
print(query,"\n")
print(f"Answer: {answer}")
print("\nSource Documents:")
for doc in source_docs:
    print(f"- : {doc.page_content[:100]}...")

Can you summerize the topic? 

Answer: The theoretical system continues to mature process, but rely on the machine itself to complete the analysis of data information. In practice, the operation method is to let the machine learn the basic concepts and content, and then give the machine enough freedom to complete a series of content learning, including concepts and content similar to the basic principles, such as tree roots Promotion of Personalized Customization Services.

Source Documents:
- : # 1. Introduction...
- : machine learning theory will be continuously strengthened, and the degree of refinement of the conte...
- : process presents a certain regularity, and the learning content is more systematic [1]...
- : # 4. Research on Machine Learning Development # 4.1. Theoretical System Continues to Mature...
- : process, but rely on the machine itself to complete the analysis of data information. In practice, t...
- : Promotion of Personalized Customization Services...


In [8]:
# Extract documents
path = r"C:\Users\Sayan Maity\OneDrive\Desktop\chat_pdf\Research_on_Machine_Learning_and_Its_Algorithms_an.pdf"
documents = extract_documents(path)

# Inspect the first document to understand its structure
first_document = documents[0]
print(first_document.__dict__)



Started parsing the file under job_id 28bd0b85-5696-4ae2-b668-27ba0ac01195
{'id_': '2b05c1a6-8fa3-4273-b168-e2bdabada1a6', 'embedding': None, 'metadata': {}, 'excluded_embed_metadata_keys': [], 'excluded_llm_metadata_keys': [], 'relationships': {}, 'text': '# Journal of Physics: Conference Series\n\nPAPER &#8226; OPEN ACCESS\n\nResearch on Machine Learning and Its Algorithms and Development\n\nTo cite this article: Wei Jin 2020 J. Phys.: Conf. Ser. 1544 012003\n\nView the article online for updates and enhancements.\n\nIOP ebooks\n\nBringing together innovative digital publishing with leading authors from the global scientific community.\n\nStart exploring the collection-download the first chapter of every title for free.\n\nThis content was downloaded from IP address 158.46.154.149 on 03/06/2020 at 13:35', 'mimetype': 'text/plain', 'start_char_idx': None, 'end_char_idx': None, 'text_template': '{metadata_str}\n\n{content}', 'metadata_template': '{key}: {value}', 'metadata_seperator': 